# Machine Learning in Business Project

In this project for the OilyGiant mining, my task is to find the best place for a new well.
The Steps i will make to choose the location:
1. Collect the oil well parameters in the selected region: oil quality and volume of reserves;
2. Build a model for predicting the volume of reserves in the new wells;
3. Pick the oil wells with the highest estimated values;
4. Pick the region with the highest total profit for the selected oil wells, after risk evaluation.


In [2]:
#Loading all the libraries i will use:
import pandas as pd
import numpy as np
import datetime
import math 
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore")
import requests 
import io
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error



## Data Preparation 

In [4]:
# Downloading the csv file from my GitHub account

url1 = "https://raw.githubusercontent.com/yoav-karsenty/Machine-learning-in-business---OilyGiant-mining/main/geo_data_0.csv"
download1 = requests.get(url1).content
url2 = "https://raw.githubusercontent.com/yoav-karsenty/Machine-learning-in-business---OilyGiant-mining/main/geo_data_1.csv"
download2 = requests.get(url2).content
url3 = "https://raw.githubusercontent.com/yoav-karsenty/Machine-learning-in-business---OilyGiant-mining/main/geo_data_2.csv"
download3 = requests.get(url3).content

# Reading the downloaded content and turning it into a pandas dataframe

g0 = pd.read_csv(io.StringIO(download1.decode('utf-8')))
g1 = pd.read_csv(io.StringIO(download2.decode('utf-8')))
g2 = pd.read_csv(io.StringIO(download3.decode('utf-8')))


# Exploring the data
g1.head(7)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147


In [5]:
#Exploring the dataset
g0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
g0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
g0.duplicated().sum()

0

No missing values or duplicates. We can nove on

In [6]:
#Dropping the id column from the datasets, it will interfere with my model training 
g0.drop(['id'],axis = 1,inplace = True)
g1.drop(['id'],axis = 1,inplace = True)
g2.drop(['id'],axis = 1,inplace = True)


Next, i want to get predictions for volume of reserves for new wells in each region. 
In order to do that i will train a LinearRegression model. 

## Model Training 

In [7]:
features0 = g0.drop(['product'],axis =1) #Getting the features and target from the dataset

target0 =g0['product']

features_train0, features_valid0, target_train0,target_valid0 = train_test_split(
features0, target0, test_size=0.25, random_state=12345) # spliting 25% of data to make validation set

model = LinearRegression()

model.fit(features_train0,target_train0) # training model on training set
predictions_0 = model.predict(features_valid0) # getting model predictions on validation set
pred =pd.Series(predictions_0, index = target_valid0.index)

In [8]:
features1 = g1.drop(['product'],axis =1) #Getting the features and target from the dataset

target1 =g1['product']

features_train1, features_valid1, target_train1,target_valid1 = train_test_split(
features1, target1, test_size=0.25, random_state=12345) # splitting 25% of data to make validation set

model = LinearRegression()

model.fit(features_train1,target_train1) # training model on training set
predictions_1 = model.predict(features_valid1) # getting model predictions on validation set
pred1 =pd.Series(predictions_1, index = target_valid1.index)

In [9]:
features2 = g2.drop(['product'],axis =1)#Getting the features and target from the dataset

target2 =g2['product']

features_train2, features_valid2, target_train2,target_valid2 = train_test_split(
features2, target2, test_size=0.25, random_state=12345) # split 25% of data to make validation set

model = LinearRegression()

model.fit(features_train2,target_train2) # train model on training set
predictions_2 = model.predict(features_valid2) # get model predictions on validation set
pred2 =pd.Series(predictions_2, index = target_valid2.index)

Next i will build a function that will Print the average volume of predicted reserves model RMSE.


In [10]:
#i will use these lists to create a DataFrame later on
rmse_score = []
names = []
avg_volume = []

In [11]:
def rmse(df,target_valid,predictions_valid):
    
    result = mean_squared_error(target_valid,predictions_valid)**0.5
    name =[x for x in globals() if globals()[x] is df][0]
    print("Region: %s" % name)
    print("RMSE of the linear regression model on the validation set:", result)
    fake_predictions = pd.Series(target_valid.mean(), index=target_valid.index) 

    rmse = mean_squared_error(target_valid,fake_predictions)**0.5
    rmse_score.append(result)
    names.append(name)
    avg_volume.append(predictions_valid.mean())
    print('RMSE of fake model:', rmse)
    print("Average volume of predicted reserves:" ,predictions_valid.mean()  )

    print('---'*10)

In [12]:
#creating a list of parameters, i will use that to apply the functoin 
list_target_preds = [[g0,target_valid0,predictions_0],[g1,target_valid1,predictions_1],[g2,target_valid2,predictions_2]]

In [13]:
#Applying the function 
for i in list_target_preds:
    rmse(i[0],i[1],i[2])

Region: g0
RMSE of the linear regression model on the validation set: 37.5794217150813
RMSE of fake model: 44.28602687855358
Average volume of predicted reserves: 92.59256778438035
------------------------------
Region: g1
RMSE of the linear regression model on the validation set: 0.893099286775617
RMSE of fake model: 46.0212449226281
Average volume of predicted reserves: 68.728546895446
------------------------------
Region: g2
RMSE of the linear regression model on the validation set: 40.02970873393434
RMSE of fake model: 44.902084376898294
Average volume of predicted reserves: 94.96504596800489
------------------------------


In [14]:

model_scores = pd.DataFrame(zip(names, rmse_score,avg_volume),
               columns =['Name', 'rmse_score','avg_vol'])
model_scores

,Name,rmse_score,avg_vol
0,g0,37.579422,92.592568
1,g1,0.893099,68.728547
2,g2,40.029709,94.965046


It looks like the g1 region has the lowest average volume of predicted reserves, but it has the best rmse score by far - 0.89. I will need to investigate more to reach more conclusive conclusions.   

Next, i want to Prepare the data for profit calculation. i will do the following:
1. Store all key values for calculations in separate variables.
2. Calculate the volume of reserves sufficient for developing a new well without losses. 
3. Compare the obtained value with the average volume of reserves in each region.
4. Write a function to calculate profit from a set of selected oil wells and model predictions:

the conditions i should consider:
1. When exploring the region, a study of 500 points is carried with picking the best 200 points for the profit calculation.
2. The budget for development of 200 oil wells is 100 USD million.
3. One barrel of raw materials brings 4.5 USD of revenue The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels).



## Calculating Profits 

In [15]:
#Storing all key values for calculations in separate variables

#Number of wells to be developed.
WELLS = 200

#Number of wells to sample in bootstrapping.
N_BOOT = 500

#Revenue (in USD) per one unit of product.
PRODUCT_REV = 4500

#Total well development budget.
BUDJET = 100000000

#Budget per well.
BUDJUET_PER_WELL = BUDJET / WELLS

In [27]:
#Calculating minimum volume of reserves sufficient for developing a new well without losses.
print(f"Minimum volume of reserves to avoid loss: {round((BUDJUET_PER_WELL/PRODUCT_REV), 2)}")

#Addding column to model_scores for difference between each regions average volume and minimum required volume.
model_scores['Vol. Diff.'] = model_scores['avg_vol'] - round((BUDJUET_PER_WELL/PRODUCT_REV), 2)

display(model_scores)




Minimum volume of reserves to avoid loss: 111.11


,Name,rmse_score,avg_vol,Vol. Diff.
0,g0,37.579422,92.592568,-18.517432
1,g1,0.893099,68.728547,-42.381453
2,g2,40.029709,94.965046,-16.144954


In [28]:
# A function to calculate profit from a set of selected oil wells and model predictions:
def profit(target, pred, count):
    pred_sorted = pred.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    return (PRODUCT_REV * selected.sum()) - BUDJUET_PER_WELL*count



In [29]:
#Applying the function
profit(target_valid0,pred,WELLS)

33208260.43139851

In [30]:
profit(target_valid1,pred1,WELLS)

24150866.966815114

In [31]:
profit(target_valid2,pred2,WELLS)

27103499.635998324

We can see the best region for oil wells' development in terms of revenue is g0 with 33,208,260.43. 
next i want to calculate risks and profit for each region, given the next condition:
1. After the risk evaluation, keep only the regions with the risk of losses lower than 2.5%. From the ones that fit the criteria, the region with the highest average profit should be selected. 

In order to get a 95% confidence interval, i wiil use the bootstrap technique.
I will build a function that will do the following for every region:
1. Loop 1000 times
2. For each iteration take a sample with size 500, replace=True
3. For each sample take the top 200 wells based on the prediction of the production volume 
4. call the profiy function(target_subsample, pred_subsample, 200), where the count is equal to 200.
5. Save the profit for each sample in the array. The size of the array would be 1000, which is the number of loops.
6. Compute and return statistics from this array: mean, confidence interval, risk of losses.

## Bootstrapping and Risk Evaluation 

In [32]:
#Creating a random variable i will us in the sampeling process  
state = np.random.RandomState(12345)
    


In [35]:

def boot(target,predicts):
    values = []
    for i in range(1000):    
        target_subsample = target.sample(n=500, replace = True,random_state = state) # sample with 500 random wells taken from. the target
        predict_subsamples = predicts[target_subsample.index] # # sample values by indexes from target subsample tken from predictions 
        values.append(profit(target_subsample,predict_subsamples,200)) # #calling the revenue function
        
    values = pd.Series(values)
    v_mean = values.mean() # AVG Profit 

    #creating a 97.5% confidence interval

    lower =  values.quantile(0.025) # lower 2.5%
    upper = values.quantile(0.975) # upper 2.5%
 
    risk = pd.DataFrame(data = values, columns=['val']) #Creating a df out of the calues Series so i could use query
   
    perc =  risk.query('val > val.quantile(0.025) and val < val.quantile(0.975)') 
    print("Average revenue:", v_mean)
    print("2.5% quantile:", lower)
    print("97.5% quantile:", upper)

    print ("Risk percentage:",risk[risk['val'] < 0].count() / 1000)


In [36]:
#Applying the boot function
boot(target_valid0,pred)

Average revenue: 4215633.061676807
2.5% quantile: -1194993.4941164043
97.5% quantile: 9470399.88723111
Risk percentage: val    0.06
dtype: float64


In [37]:
boot(target_valid1,pred1)

Average revenue: 5068142.889989467
2.5% quantile: 1102361.1441505544
97.5% quantile: 9400891.833443346
Risk percentage: val    0.008
dtype: float64


In [38]:
boot(target_valid2,pred2)

Average revenue: 4242124.880991598
2.5% quantile: -949202.6843301129
97.5% quantile: 9328900.21685064
Risk percentage: val    0.049
dtype: float64


## Conclusion

Given the risk criteria (risk got to be lower than 2.5% ), the only region that meets the criteria is g1. with 0% risk.  